### experimenting with scipy minimization vs. statsmodel minimization

In [66]:
from scipy.optimize import minimize
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [77]:
# need an imaginary dataset
x_obs = np.arange(-5,5,0.5)

In [78]:
# make it a wacky 4th order polynomial (we know that b4=1 and b3=3 in reality)
y_exact = x_obs**4 + 3*x_obs**3
shape(y_exact)

(20,)

In [88]:
# add a bunch of error to y_exact, to make it a more realistic dataset
y_obs = y_exact + np.random.rand(20)*10
shape(y_obs)

(20,)

In [90]:
# convert to a tidy dataframe
d = {'y': y_obs, 'x':x_obs}
our_data = pd.DataFrame(d)
print shape(our_data)
our_data.head()

(20, 2)


,x,y
0,-5.0,250.503793
1,-4.5,143.735482
2,-4.0,72.892716
3,-3.5,27.551131
4,-3.0,2.807594


In [91]:
# define a function to be the data model - here, we've magically guessed that the model should be of the form:
# y = b4*x^4 + b3*x^3 --> just need to fit for the coefficients
def our_model(x, coeffs): 
    return coeffs[0]*x**4 + coeffs[1]*x**3

In [92]:
# define a function to calculate the sum of squared residuals (RSS) for any given iteration
# three args: 
# 1 - the current "guess" at best fitted coefficients
# 2 - the underlying model that we're trying to fit
# 3 - the dataset we're using
def residuals(coeff_approx, model, data):
    # get estimated y value for each x point in our dataset
    y_approx = data['x'].map(lambda x: model(x, coeff_approx))
    # compute RSS
    return (y_approx - data['y']).map(lambda diff: diff**2).sum()

In [93]:
# provide a (random) initial guess for b4 and b3 parameters
b_guess = [0.5, 2.5]

In [94]:
# try to minimize using the scipy method
minimized = minimize(residuals, b_guess, args=(our_model, our_data))

In [1]:
minimized.x

NameError: name 'minimized' is not defined

In [96]:
# try the statsmodel package now...

# first, build the exponential terms right into the dataframe
our_data['x4'] = our_data['x'].map(lambda x: x**4)
our_data['x3'] = our_data['x'].map(lambda x: x**3)

# too easy!
stat_fit = smf.ols(formula='y ~ x4 + x3', data=our_data).fit()
print stat_fit.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.024e+04
Date:                Thu, 06 Oct 2016   Prob (F-statistic):           8.72e-33
Time:                        12:55:29   Log-Likelihood:                -45.182
No. Observations:                  20   AIC:                             96.36
Df Residuals:                      17   BIC:                             99.35
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      4.8461      0.703      6.891      0.0